1. 先抓每篇文章的terms
2. 把每篇文章的terms frequency(TF)偷記下來後面用
3. 把所有Terms存在一個大Dict
4. 大Dict裡面要有index, term內容, document frequency(DF)
5. 弄consine similarity

In [1]:
%run TextPRocessor.ipynb

import os
import math

In [2]:
doc1 = "IRTM/1.txt"
doc2 = "IRTM/2.txt"
doc3 = "IRTM/3.txt"

dataDir = "Data/"
outputDir = "Output/"
dicionary_OutputFileName = "dictionary.txt"

if not os.path.isdir(outputDir): os.makedirs(outputDir)

In [3]:
def listFiles(dir_path):
    fList = []
    for dirPath, dirName, fileNames in os.walk(dir_path):
        fList.extend(fileNames)
    return fList

In [4]:
# save dictionary list to text file.
def saveDictionary(dictionary_list):
    
    with open(outputDir+dicionary_OutputFileName, "w") as opf:
        
        for item in dictionary_list:
            index = str(item[0])
            term = item[1]
            df = str(item[2])
            # write to file
            opf.write(index+", "+term+", "+df)
            opf.write("\n")
            
        opf.close()

In [5]:
# sort the dictionary by term(ascending order)
# add the index into tuple

def sortDictionary(dictionary_Dict):
    tempList = sorted(dictionary_Dict.items(), key = lambda x : x[0])
    index = 0
    outputList = list()
    for item in tempList:
        term = item[0]
        df = item[1]
        outputList.append((index, term, df))
        index+=1
        
    return outputList

In [6]:
# for using and efficient purpose, turn the list to dict type
def transformDictionaryListToDict(dictionaryList):
    outputDict = dict()
    for item in dictionaryList:
        index = item[0]
        term = item[1]
        df = item[2]
        outputDict[term] = (index, df)
    return outputDict

In [7]:
def calculateIDF(df, totalDocNumbers):
    ratio = totalDocNumbers / df
    return math.log10(ratio)

In [8]:
def calculateTFIDF(documentTF_dict, dictionary_dict, totalDocNumbers):
    outputDict = dict()
    for term, termFrequency in documentTF_dict.items():
        termObj_in_dictionary =  dictionary_dict[term]
        termIndex = termObj_in_dictionary[0]
        df = termObj_in_dictionary[1]
        idf = calculateIDF(df, totalDocNumbers)
        tfidf = termFrequency * idf
        outputDict[termIndex] = tfidf
    return outputDict

In [9]:
def saveDocTFIDFInformation(docID, tfidf_info):
    with open(outputDir+docID+".txt", "w") as opf:
        tfidf_list = sorted(tfidf_info.items(), key=lambda x : x)
        for item in tfidf_list:
            index = str(item[0])
            tfidf = str(item[1])
            opf.write(index +", "+tfidf + "\n")
        
        opf.close()

In [11]:
# terms = getTermsDict(doc2)
# print(terms)

dictionary = dict() # total term Dictionary
tfPool = dict()

fileList = listFiles(dataDir)
for file in fileList:
    fileName = file.split('.')[0] # get only number
    terms = getTermsDict(dataDir + file)
    tfPool[fileName] = terms # update TF pool
    
    for key, value in terms.items(): # update Document Frequency(i.e., DF)
        if key in dictionary:
            dictionary[key] += 1
        else:
            dictionary[key] = 1
            
            
sortedDictionary = sortDictionary(dictionary) # sort and change to list.

saveDictionary(sortedDictionary)

dictionaryDict = transformDictionaryListToDict(sortedDictionary)

docTotalNumbers = len(fileList)

for key, value in tfPool.items(): # key is docID, value is TF dict
    doc_tfidf_Info = calculateTFIDF(value, dictionaryDict, docTotalNumbers) # a dict of document tf-idf information.
    saveDocTFIDFInformation(key, doc_tfidf_Info)